In [ ]:
import pandas as pd
from flask import Flask, request, current_app, jsonify, send_from_directory
from pandas.io.json import json_normalize
import itertools
import operator
from collections import Counter

app = Flask(__name__)

@app.route("/filme/<nome>", methods=['POST'])
def filme(nome):
    df = pd.read_csv('IMDbmovies.csv')
    df = df[["title","original_title","actors","avg_vote","genre"]]
    df2 = df.set_index("title").loc[nome]
    return df2.to_json(orient='records'),200

@app.route("/ator/<nome>", methods=['POST'])
def ator(nome):
    df = pd.read_csv('IMDb names.csv')
    df_films = pd.read_csv('IMDbmovies.csv')
    df_films2 = df_films.set_index("imdb_title_id")
    df2 = df.set_index("name").loc[nome]
    film_list = df2["known_for_titles"].split(",")
    genre_list = []
    df_final = pd.DataFrame()

    for x in film_list:
        temp = df_films2.loc[x]
        df_final = df_final.append(temp)

        for y in temp["genre"].split(","):
            genre_list.append(y)
    df_final = df_final[["title","avg_vote","genre"]]
    jason = [{"Actor":nome},df_final.to_dict('records'), Counter(genre_list)]
    return jsonify(jason),200

@app.route("/genero/<nome>", methods=['POST'])
def genero(nome):
    df_movies = pd.read_csv('IMDbmovies.csv')
    genero = nome #simula GET genre
    gen = df_movies[['imdb_title_id','title','year','genre','avg_vote']]
    gen = gen.sort_values('avg_vote', ascending=False)
    genero1 = gen[gen['genre'].str.contains(genero, na=False)]
    return jsonify(genero1.head(25).to_dict(orient='records')),200

@app.route("/ano/<ano>", methods=['POST'])
def ano(ano):
    year = int(ano)
    movies = pd.read_csv('IMDbmovies.csv')
    filt = movies[['imdb_title_id','title','year','genre','avg_vote']]
    filt = filt[(filt['year'] == year)].sort_values('avg_vote', ascending=False).head(25)
    return filt.to_json(orient='records')


app.run(host="0.0.0.0", port=8250)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:8250/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Jan/2020 21:08:37] "POST /ano/1991 HTTP/1.1" 200 -
127.0.0.1 - - [29/Jan/2020 21:09:08] "POST /filme/Cleopatra HTTP/1.1" 200 -
127.0.0.1 - - [29/Jan/2020 21:09:44] "POST /filme/Cleopatra HTTP/1.1" 200 -
